In [99]:
import os
import pandas as pd
import requests
import random
import tqdm.notebook as tqdm
from datetime import date, datetime

usr = os.path.expanduser("~")

In [8]:
with open(os.path.join(usr, "Downloads/cuzk.secret")) as f:
    CUZK_SECRET = f.read().strip()

In [39]:
class CisloRizeni:

    def __init__(
        self,
        poradove_cislo: int,
        typ_rizeni: str = "V",
        rok: int = 2025,
        kod_pracoviste: int = 101,
    ):
        self.poradove_cislo = poradove_cislo
        self.typ_rizeni = typ_rizeni
        self.kod_pracoviste = kod_pracoviste
        self.rok = rok

    def __str__(self) -> str:
        return f"{self.typ_rizeni}-{self.poradove_cislo}/{self.rok}-{self.kod_pracoviste}"

    def __repr__(self) -> str:
        return self.__str__()

    def get_url(self) -> str:
        url = "https://api-kn.cuzk.gov.cz/api/v1/Rizeni/Vyhledani"
        params = [
            f"TypRizeni={self.typ_rizeni}",
            f"Cislo={self.poradove_cislo}",
            f"Rok={self.rok}",
            f"KodPracoviste={self.kod_pracoviste}",
        ]
        params = "&".join(params)
        return url + "?" + params

In [40]:
random.seed(0)
cisla_rizeni = [
    CisloRizeni(n) for n in range(42000, 46000)
]

cisla_rizeni = random.sample(cisla_rizeni, k=480)

In [48]:
headers = {
    "ApiKey": CUZK_SECRET
}

response_51598 = requests.get(CisloRizeni(51598).get_url(), headers=headers)

In [63]:
response_51598.json()

{'data': [{'id': 104131983010,
   'typRizeni': 'V',
   'poradoveCislo': 51598,
   'rok': 2025,
   'kodPracoviste': 101,
   'datumPrijeti': '2025-08-13T09:00:18',
   'stav': 'U nemovitostí byla vyznačena plomba',
   'stavUhrady': 'U',
   'provedeneOperace': [{'nazev': 'Založení řízení',
     'datumProvedeni': '2025-08-13T09:00:44'},
    {'nazev': 'Zaplombování', 'datumProvedeni': '2025-08-13T09:00:45'}],
   'navazanaRizeni': None,
   'poznamky': ['U řízení typu V se neposkytují navázaná řízení']}],
 'zpravy': [],
 'aktualnostDatK': '2025-08-26T19:00:00',
 'provedenoVolani': 2}

In [50]:
responses = [
    requests.get(cislo_rizeni.get_url(), headers=headers)
    for cislo_rizeni in tqdm.tqdm(cisla_rizeni)
]

  0%|          | 0/480 [00:00<?, ?it/s]

In [51]:
assert all(r.status_code == 200 for r in responses)

In [59]:
data = [r.json()["data"][0] for r in responses]

In [61]:
pd.DataFrame(data)["stav"].value_counts()

stav
Odesláno vyrozumění o provedeném vkladu          436
Řízení bylo zastaveno                              8
Odeslání informace o vyznačení plomby              8
Řízení bylo ukončeno                               7
Řízení bylo přerušeno                              4
Probíhá zpracování                                 4
Řízení bylo uzavřeno jako mylné                    3
Vydáno rozhodnutí o částečném povolení vkladu      2
Obeslání účastníků řízení                          2
Zaslána výzva k zaplacení správního poplatku       2
Vydáno rozhodnutí o povolení vkladu                2
U nemovitostí byla vyznačena plomba                1
Vydáno rozhodnutí o zamítnutí vkladu               1
Name: count, dtype: int64

In [93]:
def is_candidate(rizeni: dict) -> bool:
    correct_stav = rizeni["stav"] == "Odesláno vyrozumění o provedeném vkladu"
    operace = [o["nazev"] for o in rizeni["provedeneOperace"]]
    no_informace_o_vyznaceni_op = "Informace o vyznačení plomby" not in operace
    zaplombovani_op = "Zaplombování" in operace

    return correct_stav and no_informace_o_vyznaceni_op and zaplombovani_op

In [94]:
test = response_51598.json()["data"][0]
test["stav"] = "Odesláno vyrozumění o provedeném vkladu"
assert is_candidate(test)

In [95]:
candidates = [rizeni for rizeni in data if is_candidate(rizeni)]

In [96]:
len(candidates)

18

In [102]:
def get_time_delta(rizeni):
    start = [
        op["datumProvedeni"] for op in rizeni["provedeneOperace"]
        if op["nazev"] == "Založení řízení"
    ][0]

    end = [
        op["datumProvedeni"] for op in rizeni["provedeneOperace"]
        if op["nazev"] == "Provedení vkladu"
    ][0]

    start = datetime.strptime(start, "%Y-%m-%dT%H:%M:%S")
    end = datetime.strptime(end, "%Y-%m-%dT%H:%M:%S")
    return (end - start).days

In [107]:
mean_delta = pd.Series(get_time_delta(rizeni) for rizeni in candidates).mean()
print(f"Průměrná doba od založení řízení do provedení vkladu: {mean_delta:.1f} dní")

Průměrná doba od založení řízení do provedení vkladu: 12.2 dní


In [110]:
pd.Series(get_time_delta(rizeni) for rizeni in candidates).describe()

count    18.000000
mean     12.222222
std       7.008865
min       2.000000
25%       7.500000
50%      11.500000
75%      13.750000
max      29.000000
dtype: float64